# This example shows a pre-trained Mnist10 + MLP benchmark setting and evaluates Influence Function (CG) algorithm by LDS.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carolinef35/dattri/blob/colab_examples/examples/quickstart/influence_function_lds.ipynb)

Note: The installation block in the notebook is specifically designed for Google Colab and the use cases in this notebook. Standard installation instructions can me found in the [README](https://github.com/TRAIS-Lab/dattri/blob/main/README.md#quick-start).

In [ ]:
!pip install dattri

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.8 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.11-py3-none-any.whl size=5595886 sha256=040d236e342933211d5e6aa0d420205d4acc00910f6c13a15c2ac289abe8733c
  Stored in directory: /root/.cache/pip/wheels/f4/ad/93/a7042fe12668827574927ade9deec7f29aad2a1001b1501882
Successfully built pretty-midi


Import libraries needed to run code.

Note: If ""----- WARNING: CUDA devices not detected. This will cause the model to run very slow! -----" message appears, change your runtime type to GPU by going to Runtime -> Change runtime type and selecting 'GPU'.

In [ ]:
import argparse

import torch
from torch import nn
from torch.utils.data import DataLoader

from dattri.algorithm.influence_function import IFAttributorCG
from dattri.benchmark.load import load_benchmark
from dattri.metric import lds
from dattri.task import AttributionTask

LDS Score: used to evaluate the overall performance of a data attribution method.

* A score near 1 means the attribution method accurately predicts the model's response to data changes



In [ ]:

# initialize argument parser to handle command-line arguments
parser = argparse.ArgumentParser()
# dynamically set device to 'cuda' if available, otherwise 'cpu'
parser.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
args = parser.parse_args([])

# download the pre-trained benchmark
# includes some trained model and ground truth
model_details, groundtruth = load_benchmark(
    model="mlp", dataset="mnist", metric="lds"
)

# define a functional loss function 'f' that calculates CrossEntropyLoss
# takes model parameters and a data-target pair (image, label) as input
def f(params, data_target_pair):
    image, label = data_target_pair
    loss = nn.CrossEntropyLoss()
    # apply the model with given parameters to the image.
    yhat = torch.func.functional_call(model_details["model"], params, image)
    return loss(yhat, label.long())

# initialize the AttributionTask with the model, loss function, and model checkpoints
task = AttributionTask(
    model=model_details["model"].to(args.device),
    loss_func=f,
    checkpoints=model_details["models_full"][0],  # use one full model checkpoint
)

# initialize the IFAttributorCG (Influence Function Attributor using Conjugate Gradient)
# requires the task, device, regularization parameter, and max iterations
attributor = IFAttributorCG(
    task=task, device=args.device, regularization=5e-3, max_iter=10
)
# cache the training data using a DataLoader
# pre-processes/stores training data for attribution
attributor.cache(
    DataLoader(
        model_details["train_dataset"],
        batch_size=5000,
        sampler=model_details["train_sampler"],
    )
)

# perform attribution without gradient calculation (inference mode)
with torch.no_grad():
    # calculate influence scores of training data on test data
    score = attributor.attribute(
        DataLoader(
            model_details["train_dataset"],
            batch_size=5000,
            sampler=model_details["train_sampler"],
        ),
        DataLoader(
            model_details["test_dataset"],
            batch_size=5000,
            sampler=model_details["test_sampler"],
        ),
    )

# calculate the LDS score
lds_score = lds(score, groundtruth)[0]
# print the mean of the non-null LDS scores
print("lds:", torch.mean(lds_score[~torch.isnan(lds_score)]))

100%|██████████| 9.91M/9.91M [00:00<00:00, 14.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.62MB/s]
calculating gradient of test set...: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it]
/usr/local/lib/python3.12/dist-packages/dattri/metric/metrics.py:58: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearmanr(sum_scores[:, i], gt_values[:, i]).correlation,


lds: tensor(0.2594)


/usr/local/lib/python3.12/dist-packages/dattri/metric/metrics.py:68: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearmanr(sum_scores[:, i], gt_values[:, i]).pvalue,
